In [ ]:
!gunzip listings.csv.gz

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer


listings_df = pd.read_csv('listings.csv')
listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6713005,https://www.airbnb.com/rooms/6713005,20240503175146,2024-05-04,previous scrape,Large one bedroom,You'll share an apartment with two wonderful y...,NaN,https://a0.muscache.com/pictures/84725702/12f8...,35144920,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
1,922527181364298444,https://www.airbnb.com/rooms/922527181364298444,20240503175146,2024-05-03,city scrape,New Flushing Modern Apartment,This unique place has a style all its own.,NaN,https://a0.muscache.com/pictures/f5d74ed9-0b15...,420975523,...,5.00,4.45,5.00,NaN,t,3,3,0,0,1.10
2,794696802406920855,https://www.airbnb.com/rooms/794696802406920855,20240503175146,2024-05-04,previous scrape,Lovely studio in Hell's Kitchen!,Amazing location in the heart of Hell’s Kitche...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,43305568,...,5.00,4.91,4.91,NaN,f,1,1,0,0,0.69
3,54294086,https://www.airbnb.com/rooms/54294086,20240503175146,2024-05-03,city scrape,Luxury 1- bedroom rental unit with Hot tub,Relax with the whole family at this peaceful p...,NaN,https://a0.muscache.com/pictures/9613630a-19c4...,440355290,...,NaN,NaN,NaN,NaN,t,1,0,1,0,NaN
4,1039262508734112133,https://www.airbnb.com/rooms/1039262508734112133,20240503175146,2024-05-04,city scrape,Private room close to LGA,We are located 8 minutes from la guardia airpo...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,430854617,...,4.74,4.58,4.63,Exempt,t,3,0,3,0,3.77


In [ ]:

listings_df = listings_df.dropna(subset=['price'])

# Converting price to numerical
listings_df['price'] = listings_df['price'].replace('[\$,]', '', regex=True).astype(float)


<ipython-input-3-17908f436a59>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_df['price'] = listings_df['price'].replace('[\$,]', '', regex=True).astype(float)


In [ ]:
# Calculating the IQR and identifying the outliers
Q1 = listings_df['price'].quantile(0.25)
Q3 = listings_df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Removing the outliers
listings_df = listings_df[(listings_df['price'] >= lower_bound) & (listings_df['price'] <= upper_bound)]

In [ ]:
listings_df = listings_df[['room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights',
    'maximum_nights', 'number_of_reviews','review_scores_rating', 'reviews_per_month', 'host_is_superhost', 'host_listings_count','instant_bookable','host_since',
             'amenities', 'price']]

listings_df.head()

,room_type,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,reviews_per_month,host_is_superhost,host_listings_count,instant_bookable,host_since,amenities,price
1,Entire home/apt,4,1.0,1.0,2.0,30,365,11,4.91,1.10,f,6.0,t,2021-08-31,"[""Hair dryer"", ""Air conditioning"", ""Essentials...",202.0
3,Private room,2,1.0,1.0,1.0,30,365,0,NaN,NaN,f,1.0,t,2022-01-13,"[""Exterior security cameras on property"", ""BBQ...",342.0
4,Private room,2,1.0,1.0,1.0,1,365,19,4.79,3.77,f,3.0,t,2021-11-07,"[""Smoke alarm"", ""Air conditioning"", ""First aid...",105.0
5,Private room,2,1.0,1.0,1.0,30,365,0,NaN,NaN,f,1.0,f,2023-07-21,"[""Smoke alarm"", ""Air conditioning"", ""Fire exti...",51.0
6,Private room,1,1.0,1.0,1.0,30,90,4,4.75,0.06,f,1.0,t,2015-07-07,"[""Hair dryer"", ""Air conditioning"", ""Luggage dr...",125.0


In [ ]:
amenities_to_check = [
    'Dining table', 'Shower gel', 'Hot water', 'Body soap', 'Oven',
    'Dishes and silverware', 'Laundromat nearby', 'Bathtub',
    'Exterior security cameras on property', 'Shampoo', 'Dedicated workspace',
    'Smoke alarm', 'Stove', 'Hot water kettle', 'Pets allowed',
    'Private entrance', 'Long term stays allowed', 'Heating',
    'Carbon monoxide alarm', 'Room-darkening shades', 'Cleaning products',
    'Refrigerator', 'Fire extinguisher', 'Hangers', 'Hair dryer',
    'Iron', 'Microwave', 'Self check-in', 'Freezer', 'Luggage dropoff allowed',
    'Dishwasher', 'Baking sheet', 'Wine glasses', 'Bed linens', 'Lockbox',
    'Washer', 'Conditioner', 'Wifi', 'Coffee maker', 'Free street parking',
    'Toaster', 'Extra pillows and blankets', 'Cooking basics', 'Kitchen',
    'Air conditioning', 'TV', 'Lock on bedroom door', 'Coffee', 'Essentials',
    'First aid kit'
]

In [ ]:

for amenity in amenities_to_check:
    column_name = amenity.lower().replace(' ', '_').replace('[', '').replace(']', '')
    listings_df[column_name] = listings_df['amenities'].apply(lambda x: 1 if amenity in x else 0)


print(listings_df.head())

         room_type  accommodates  bathrooms  bedrooms  beds  minimum_nights  \
1  Entire home/apt             4        1.0       1.0   2.0              30   
3     Private room             2        1.0       1.0   1.0              30   
4     Private room             2        1.0       1.0   1.0               1   
5     Private room             2        1.0       1.0   1.0              30   
6     Private room             1        1.0       1.0   1.0              30   

   maximum_nights  number_of_reviews  review_scores_rating  reviews_per_month  \
1             365                 11                  4.91               1.10   
3             365                  0                   NaN                NaN   
4             365                 19                  4.79               3.77   
5             365                  0                   NaN                NaN   
6              90                  4                  4.75               0.06   

   ... toaster  extra_pillows_and_blan

In [ ]:
# Replacing all null values in the new columns with zeros
start_index_of_new_columns = listings_df.columns.get_loc('dining_table')
cols_to_replace_nulls = listings_df.iloc[:, start_index_of_new_columns:].columns
listings_df[cols_to_replace_nulls] = listings_df[cols_to_replace_nulls].fillna(0)


listings_df.drop('amenities', axis=1, inplace=True)

listings_df.head()

,room_type,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,reviews_per_month,...,toaster,extra_pillows_and_blankets,cooking_basics,kitchen,air_conditioning,tv,lock_on_bedroom_door,coffee,essentials,first_aid_kit
1,Entire home/apt,4,1.0,1.0,2.0,30,365,11,4.91,1.10,...,0,1,1,1,1,1,0,0,1,0
3,Private room,2,1.0,1.0,1.0,30,365,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,Private room,2,1.0,1.0,1.0,1,365,19,4.79,3.77,...,0,0,0,1,1,0,1,0,0,1
5,Private room,2,1.0,1.0,1.0,30,365,0,NaN,NaN,...,0,0,0,1,1,0,1,0,0,0
6,Private room,1,1.0,1.0,1.0,30,90,4,4.75,0.06,...,0,1,1,1,1,1,0,1,1,0


In [ ]:
# Removing the amenities which have all NULL values for all listings
listings_df = listings_df.dropna(axis=1, how='all')
listings_df

,room_type,accommodates,bathrooms,bedrooms,beds,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,reviews_per_month,...,toaster,extra_pillows_and_blankets,cooking_basics,kitchen,air_conditioning,tv,lock_on_bedroom_door,coffee,essentials,first_aid_kit
1,Entire home/apt,4,1.0,1.0,2.0,30,365,11,4.91,1.10,...,0,1,1,1,1,1,0,0,1,0
3,Private room,2,1.0,1.0,1.0,30,365,0,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,Private room,2,1.0,1.0,1.0,1,365,19,4.79,3.77,...,0,0,0,1,1,0,1,0,0,1
5,Private room,2,1.0,1.0,1.0,30,365,0,NaN,NaN,...,0,0,0,1,1,0,1,0,0,0
6,Private room,1,1.0,1.0,1.0,30,90,4,4.75,0.06,...,0,1,1,1,1,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38194,Private room,1,1.0,1.0,0.0,30,365,0,NaN,NaN,...,0,0,0,1,0,1,1,0,1,1
38195,Private room,1,1.0,1.0,1.0,30,365,0,NaN,NaN,...,0,0,0,1,1,1,1,0,0,0
38196,Private room,2,1.0,1.0,1.0,30,31,28,4.93,1.32,...,0,0,0,0,1,0,0,0,1,0
38197,Private room,2,1.0,1.0,2.0,30,1125,134,4.48,2.08,...,0,0,0,0,1,1,1,1,1,0


In [ ]:
data = listings_df.dropna()
# Log-transforming the price
data['log_price'] = np.log(data['price'])

<ipython-input-10-6a6a600439b3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['log_price'] = np.log(data['price'])


In [ ]:
data[['room_type', 'instant_bookable', 'host_since','host_is_superhost']].head()



,room_type,instant_bookable,host_since,host_is_superhost
1,Entire home/apt,t,2021-08-31,f
4,Private room,t,2021-11-07,f
6,Private room,t,2015-07-07,f
7,Private room,f,2016-07-11,f
10,Entire home/apt,f,2021-08-12,f


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Converting binary categorical variables to 0/1
data['instant_bookable'] = data['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
data['host_is_superhost'] = data['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)

# Converting host_since to the number of days since the host started
data['host_since'] = pd.to_datetime(data['host_since'], format='%Y-%m-%d')
data['host_since_days'] = (datetime.now() - data['host_since']).dt.days

# Dropping the original host_since column
data = data.drop(['host_since'], axis=1)

<ipython-input-12-db9fc35de378>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['instant_bookable'] = data['instant_bookable'].apply(lambda x: 1 if x == 't' else 0)
<ipython-input-12-db9fc35de378>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['host_is_superhost'] = data['host_is_superhost'].apply(lambda x: 1 if x == 't' else 0)
<ipython-input-12-db9fc35de378>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [ ]:
data[['room_type', 'instant_bookable', 'host_is_superhost', 'host_since_days']].head()



,room_type,instant_bookable,host_is_superhost,host_since_days
1,Entire home/apt,1,0,1017
4,Private room,1,0,949
6,Private room,1,0,3264
7,Private room,0,0,2894
10,Entire home/apt,0,0,1036


In [ ]:
categorical_features = [
    'room_type'
]
# Applying one-hot encoding to the remaining categorical variables
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

print(data.head())

    accommodates  bathrooms  bedrooms  beds  minimum_nights  maximum_nights  \
1              4        1.0       1.0   2.0              30             365   
4              2        1.0       1.0   1.0               1             365   
6              1        1.0       1.0   1.0              30              90   
7              2        2.0       1.0   1.0              30            1125   
10             8        1.0       3.0   4.0              30            1125   

    number_of_reviews  review_scores_rating  reviews_per_month  \
1                  11                  4.91               1.10   
4                  19                  4.79               3.77   
6                   4                  4.75               0.06   
7                  57                  4.77               2.73   
10                 24                  4.71               0.77   

    host_is_superhost  ...  tv  lock_on_bedroom_door  coffee  essentials  \
1                   0  ...   1                     0

In [ ]:
data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna()
print(data.dtypes)


accommodates                int64
bathrooms                 float64
bedrooms                  float64
beds                      float64
minimum_nights              int64
                           ...   
log_price                 float64
host_since_days             int64
room_type_Hotel room         bool
room_type_Private room       bool
room_type_Shared room        bool
Length: 68, dtype: object


In [ ]:
bool_columns = data.select_dtypes(include=['bool']).columns
print("Boolean columns:")
print(bool_columns)

Boolean columns:
Index(['room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')


In [ ]:
data[bool_columns] = data[bool_columns].astype(int)

In [ ]:
# Defining the features and target
X = data.drop(['price', 'log_price'], axis=1)
y = np.log(data['price'])

print("Features Data Types:")
print(X.dtypes)

print("Target Data Type:")
print(y.dtypes)

X = sm.add_constant(X)


Features Data Types:
accommodates                int64
bathrooms                 float64
bedrooms                  float64
beds                      float64
minimum_nights              int64
                           ...   
first_aid_kit               int64
host_since_days             int64
room_type_Hotel room        int64
room_type_Private room      int64
room_type_Shared room       int64
Length: 66, dtype: object
Target Data Type:
float64


In [ ]:
import numpy as np
import statsmodels.api as sm
from tqdm import tqdm

def backward_stepwise_selection(X, y, criteria='aic'):
    remaining = list(X.columns)
    best_score = np.inf
    best_model = None


    for _ in tqdm(range(len(X.columns)), desc='Evaluating models'):
        scores_with_candidates = []
        for candidate in remaining:
            try:
                model = sm.OLS(y, X[remaining].drop(candidate, axis=1)).fit()
                if criteria == 'aic':
                    score = model.aic
                elif criteria == 'bic':
                    score = model.bic
                elif criteria == 'adj_r2':
                    score = -model.rsquared_adj
                else:
                    raise ValueError("Criteria should be 'aic', 'bic', or 'adj_r2'.")
                scores_with_candidates.append((score, candidate, model))
            except np.linalg.LinAlgError:
                continue

        scores_with_candidates.sort()
        best_new_score, best_candidate, best_model_candidate = scores_with_candidates[0]

        if best_new_score < best_score:
            best_score = best_new_score
            best_model = best_model_candidate
            remaining.remove(best_candidate)
        else:
            break

    return best_model


In [ ]:
# Splitting the data into train and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Perform backward stepwise selection using AIC
best_model_aic = backward_stepwise_selection(X_train, y_train, criteria='aic')
best_model_bic = backward_stepwise_selection(X_train, y_train, criteria='bic')
best_model_adj_r2 = backward_stepwise_selection(X_train, y_train, criteria='adj_r2')

# Printing the selected variables for each criterion
selected_variables_aic = best_model_aic.model.exog_names
selected_variables_bic = best_model_bic.model.exog_names
selected_variables_adj_r2 = best_model_adj_r2.model.exog_names

print("Selected variables based on AIC:", selected_variables_aic)
print("Selected variables based on BIC:", selected_variables_bic)
print("Selected variables based on Adjusted R-squared:", selected_variables_adj_r2)

best_model_aic.summary(), best_model_bic.summary(), best_model_adj_r2.summary()


Evaluating models:  16%|█▋        | 11/67 [01:43<08:47,  9.41s/it]


Selected variables based on AIC: ['const', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month', 'host_is_superhost', 'host_listings_count', 'instant_bookable', 'dining_table', 'shower_gel', 'hot_water', 'oven', 'bathtub', 'exterior_security_cameras_on_property', 'shampoo', 'smoke_alarm', 'stove', 'hot_water_kettle', 'pets_allowed', 'private_entrance', 'long_term_stays_allowed', 'heating', 'carbon_monoxide_alarm', 'room-darkening_shades', 'cleaning_products', 'fire_extinguisher', 'hangers', 'hair_dryer', 'microwave', 'self_check-in', 'luggage_dropoff_allowed', 'dishwasher', 'baking_sheet', 'wine_glasses', 'bed_linens', 'lockbox', 'washer', 'wifi', 'free_street_parking', 'toaster', 'kitchen', 'air_conditioning', 'tv', 'lock_on_bedroom_door', 'coffee', 'essentials', 'host_since_days', 'room_type_Private room', 'room_type_Shared room']
Selected variables based on BIC: ['const', 'accommodates',

(<class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                  price   R-squared:                       0.493
 Model:                            OLS   Adj. R-squared:                  0.490
 Method:                 Least Squares   F-statistic:                     198.3
 Date:                Thu, 13 Jun 2024   Prob (F-statistic):               0.00
 Time:                        07:42:32   Log-Likelihood:                -6683.2
 No. Observations:               10871   AIC:                         1.347e+04
 Df Residuals:                   10817   BIC:                         1.387e+04
 Df Model:                          53                                         
 Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
 ------------------------------------------

1. Comparison of Coefficient Values and Variable Inclusion
AIC Model:

Tends to include more variables because it prioritizes model fit over simplicity.
Coefficients may show smaller p-values, indicating strong significance due to the larger number of variables considered.
Can capture more nuances and interactions in the data but risks overfitting, which might not generalize well outside the training dataset.
BIC Model:

Includes fewer variables than the AIC model as it introduces a harsher penalty for the number of parameters.
This model is generally preferred when the primary concern is overfitting and when the aim is to simplify the model for easier interpretation and better generalization.
Coefficients in the BIC model may be slightly larger in magnitude compared to the AIC model for the same variables, as the model is less likely to attribute effects to noise.
Adjusted R-squared Model:

A balance between complexity (number of variables) and the model's ability to explain the variance in the response variable.
Selects variables that contribute significantly to the model’s explanatory power, adjusting for the number of predictors used.
Coefficients might be similar in significance to those in the AIC model but fewer in number, similar to the BIC approach.
2. Specific Observations from Your Dataset
Considering the Airbnb listings data for NYC, some variables are consistently significant across all models, like accommodates, bedrooms, review_scores_rating, etc. These factors are known to heavily influence rental pricing directly.

Variable Differences:

More nuanced variables like dining_table, shower_gel, or oven appear in the AIC model but might be absent in the BIC and adjusted R-squared models. This suggests these features, while potentially influential, are considered less critical when penalties for additional predictors are stricter.
In contrast, essentials like wifi, air_conditioning, and kitchen might show across all models, emphasizing their importance in pricing regardless of the model's complexity.
3. Model Preferences
Which Model to Prefer?

If your goal is explanatory and you're looking to understand all potential influences on pricing, the AIC model could be preferable as it captures the broadest spectrum of influencing factors.
For predictive purposes, where the model needs to perform well on unseen data, the BIC model is often more suitable because it avoids overfitting by simplifying the model.
If you seek a balance, aiming for a model that is neither too complex nor too simplistic while maintaining robustness in predictions and explanations, the Adjusted R-squared model would be ideal.
4. Coefficient Similarity
Similarity: Major variables that directly impact pricing, such as location-based features or property size (bedrooms, accommodates), will generally show similar effects across models.
Differences: Marginal or less impactful features might show greater variability in their estimated effects and significance across different models due to differing penalty strengths for the number of variables.
In conclusion, the choice of model largely depends on your specific needs—whether it is understanding, predicting, or balancing both. Each model provides a different lens through which to view the influences on Airbnb prices in NYC, and the most appropriate choice will align with your project's goals and the specific complexities of your dataset.






